# Distributional Forecasting Pipeline

**From raw data to risk-aware decisions with uncertainty quantification.**

This notebook demonstrates the complete methodology:

```
Raw Data → Feature Extraction → Model Training → PDF Parameters → E[X] & Risk → Decisions
```

Traditional ML predicts point estimates. **Distributional regression** predicts entire probability distributions, enabling:
- Uncertainty quantification at every prediction
- Risk measures (VaR, CVaR) directly from forecasts
- Optimal position sizing (Kelly criterion)
- Proper evaluation via scoring rules (CRPS)

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats as scipy_stats, optimize

import temporalpdf as tpdf

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11

print(f"temporalpdf version: {tpdf.__version__}")

---

## Step 1: Load Raw Data

We use real Nordic stock returns (2014-2016). In production, this would be your target variable - whatever you're forecasting.

In [ ]:
# Load real market data
df = pd.read_csv('../data/sample_returns.csv')
df['ASOFDATE'] = pd.to_datetime(df['ASOFDATE'])

print(f"Loaded {len(df):,} observations")
print(f"Companies: {df['COMPANYNAME'].nunique()}")
print(f"Date range: {df['ASOFDATE'].min().date()} to {df['ASOFDATE'].max().date()}")
print()
print("Sample:")
df.head()

In [ ]:
# Target distribution: forward returns
returns = df['target'].dropna().values

print("Target Statistics (% returns):")
print(f"  Mean:     {np.mean(returns):7.4f}%")
print(f"  Std:      {np.std(returns):7.4f}%")
print(f"  Skewness: {scipy_stats.skew(returns):7.4f}")
print(f"  Kurtosis: {scipy_stats.kurtosis(returns):7.4f} (excess)")
print(f"  Min:      {np.min(returns):7.2f}%")
print(f"  Max:      {np.max(returns):7.2f}%")

# Histogram
plt.figure(figsize=(10, 5))
plt.hist(returns, bins=60, density=True, alpha=0.7, color='steelblue', edgecolor='white')
plt.axvline(np.mean(returns), color='red', ls='--', lw=2, label=f'Mean = {np.mean(returns):.2f}%')
plt.xlabel('Return (%)')
plt.ylabel('Density')
plt.title('Distribution of Forward Returns')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print("\nNote: Heavy tails and slight negative skew - typical of equity returns.")
print("A Normal distribution would underestimate tail risk.")

---

## Step 2: Extract Rolling Coefficients (Features)

For each prediction point, we extract **distributional features** from the historical window:
- Rolling mean (location)
- Rolling volatility (scale)
- Rolling skewness (asymmetry)
- Volatility trend (is vol increasing or decreasing?)

These become inputs to the model that predicts distribution parameters.

In [ ]:
# Extract rolling coefficients for one company
company = df[df['COMPANYNAME'] == df['COMPANYNAME'].iloc[0]].copy()
company = company.sort_values('ASOFDATE').reset_index(drop=True)

print(f"Company: {company['COMPANYNAME'].iloc[0]}")
print(f"Observations: {len(company)}")

In [ ]:
# Configure coefficient extraction
config = tpdf.ExtractionConfig(
    value_column='target',
    time_column='ASOFDATE',
    horizon=30,              # 30-day forward horizon
    volatility_window=20,    # 20-day rolling window for vol
)

# Extract coefficients using rolling windows
extractor = tpdf.RollingCoefficientExtractor()
coefficients = extractor.extract(company, config)

print("Extracted Coefficients:")
print(coefficients[['ASOFDATE', 'mean', 'volatility', 'skewness']].dropna().head(10))

In [ ]:
# Visualize rolling coefficients over time
coef_clean = coefficients.dropna()

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

axes[0].plot(coef_clean['ASOFDATE'], coef_clean['mean'], 'b-', lw=1.5)
axes[0].axhline(0, color='gray', ls='--')
axes[0].set_ylabel('Rolling Mean (%)')
axes[0].set_title('Rolling Coefficients Over Time')
axes[0].grid(alpha=0.3)

axes[1].plot(coef_clean['ASOFDATE'], coef_clean['volatility'], 'r-', lw=1.5)
axes[1].set_ylabel('Rolling Volatility (%)')
axes[1].grid(alpha=0.3)

axes[2].plot(coef_clean['ASOFDATE'], coef_clean['skewness'], 'g-', lw=1.5)
axes[2].axhline(0, color='gray', ls='--')
axes[2].set_ylabel('Rolling Skewness')
axes[2].set_xlabel('Date')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("These rolling coefficients become FEATURES for the distributional model.")

---

## Step 3: Fit Distribution Parameters (Model Training)

In a full pipeline, you'd use **NGBoost** or **XGBoost with custom loss** to predict distribution parameters from features.

Here we demonstrate the core idea: **fit NIG parameters via Maximum Likelihood Estimation**.

The NIG (Normal Inverse Gaussian) distribution captures:
- Heavy tails (unlike Normal)
- Skewness (asymmetric risk)
- Is standard in quantitative finance (Barndorff-Nielsen, 1997)

In [ ]:
def fit_nig_mle(returns):
    """
    Fit NIG distribution parameters via Maximum Likelihood.
    
    In production, a gradient boosting model (NGBoost, XGBoost) would
    predict these parameters from features. MLE is the ground truth.
    """
    nig = tpdf.NIG()
    
    # Initial guess from moments
    x0 = np.array([
        np.mean(returns),
        np.log(np.std(returns)),
        np.log(10.0),
        0.0,
    ])
    
    def neg_log_likelihood(theta):
        mu, log_delta, log_alpha, beta_raw = theta
        delta = np.exp(log_delta)
        alpha = np.exp(log_alpha)
        beta = alpha * np.tanh(beta_raw)
        
        try:
            params = tpdf.NIGParameters(mu=mu, delta=delta, alpha=alpha, beta=beta)
            pdf_vals = np.maximum(nig.pdf(returns, 0, params), 1e-300)
            return -np.sum(np.log(pdf_vals))
        except ValueError:
            return 1e10
    
    result = optimize.minimize(neg_log_likelihood, x0, method='Nelder-Mead',
                               options={'maxiter': 2000})
    
    mu, log_delta, log_alpha, beta_raw = result.x
    delta = np.exp(log_delta)
    alpha = np.exp(log_alpha)
    beta = alpha * np.tanh(beta_raw)
    
    return tpdf.NIGParameters(mu=mu, delta=delta, alpha=alpha, beta=beta), -result.fun

In [ ]:
# Train/test split
n_train = int(len(returns) * 0.8)
train_returns = returns[:n_train]
test_returns = returns[n_train:]

print(f"Train: {len(train_returns):,} observations")
print(f"Test:  {len(test_returns):,} observations")

# Fit NIG on training data
nig = tpdf.NIG()
fitted_params, log_lik = fit_nig_mle(train_returns)

print(f"\nFitted NIG Parameters (MLE):")
print(f"  μ (location):  {fitted_params.mu:9.5f}")
print(f"  δ (scale):     {fitted_params.delta:9.5f}")
print(f"  α (steepness): {fitted_params.alpha:9.4f}")
print(f"  β (skewness):  {fitted_params.beta:9.4f}")
print(f"\nLog-likelihood: {log_lik:.2f}")

In [ ]:
# Visualize fitted distribution vs actual data
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Histogram overlay
x = np.linspace(train_returns.min() - 5, train_returns.max() + 5, 500)
axes[0].hist(train_returns, bins=50, density=True, alpha=0.6, color='steelblue', 
             edgecolor='white', label='Actual returns')
axes[0].plot(x, nig.pdf(x, 0, fitted_params), 'r-', lw=2.5, label='Fitted NIG')
axes[0].plot(x, scipy_stats.norm.pdf(x, np.mean(train_returns), np.std(train_returns)), 
             'g--', lw=2, label='Normal (baseline)')
axes[0].set_xlabel('Return (%)')
axes[0].set_ylabel('Density')
axes[0].set_title('Fitted NIG vs Actual Returns')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Right: Log scale (tail comparison)
axes[1].hist(train_returns, bins=50, density=True, alpha=0.6, color='steelblue', edgecolor='white')
axes[1].semilogy(x, nig.pdf(x, 0, fitted_params), 'r-', lw=2.5, label='Fitted NIG')
axes[1].semilogy(x, scipy_stats.norm.pdf(x, np.mean(train_returns), np.std(train_returns)), 
                 'g--', lw=2, label='Normal')
axes[1].set_xlabel('Return (%)')
axes[1].set_ylabel('Density (log scale)')
axes[1].set_title('Tail Behavior: NIG Captures Extremes')
axes[1].legend()
axes[1].grid(alpha=0.3)
axes[1].set_ylim(1e-5, 1)

plt.tight_layout()
plt.show()

print("NIG fits the heavy tails much better than Normal.")
print("This matters for risk: Normal would underestimate extreme losses.")

---

## Step 4: Construct Time-Evolving PDF

The fitted parameters define a **static** distribution. For forecasting, we need the distribution to **evolve over time**:

- Uncertainty grows as we forecast further ahead
- Volatility may mean-revert to long-run levels
- Location may drift

temporalpdf supports multiple volatility evolution models.

In [ ]:
# Add time evolution to the fitted parameters
# Scenario: Current vol is elevated, will mean-revert to long-run

long_run_vol = fitted_params.delta * 0.8  # Assume long-run is 80% of current

params_evolving = tpdf.NIGParameters(
    mu=fitted_params.mu,
    delta=fitted_params.delta,
    alpha=fitted_params.alpha,
    beta=fitted_params.beta,
    mu_drift=0.0,  # No location drift
    volatility_model=tpdf.mean_reverting(sigma_long=long_run_vol, kappa=0.05)
)

print("Time-Evolving Parameters:")
print(f"  Current vol (δ):  {fitted_params.delta:.4f}")
print(f"  Long-run vol:     {long_run_vol:.4f}")
print(f"  Mean-reversion κ: 0.05")
print()
print("Volatility Forecast:")
for t in [0, 10, 20, 30, 60]:
    vol_t = np.sqrt(nig.variance(t, params_evolving))
    print(f"  t={t:2d} days: vol = {vol_t:.4f}")

In [ ]:
# Evaluate PDF over time grid
result = tpdf.evaluate(
    nig, 
    params_evolving,
    value_range=(-30, 30),
    time_range=(0, 60),
    value_points=200,
    time_points=60,
)

print(f"PDF Matrix shape: {result.pdf_matrix.shape}")
print(f"Time grid: {result.time_grid[0]:.0f} to {result.time_grid[-1]:.0f} days")
print(f"Value grid: {result.value_grid[0]:.1f}% to {result.value_grid[-1]:.1f}%")

In [ ]:
# 3D Surface plot of time-evolving PDF
plotter = tpdf.PDFPlotter()
fig = plotter.surface_3d(result, title='Time-Evolving Forecast Distribution (60-Day Horizon)')
plt.show()

print("The PDF spreads out over time as uncertainty grows.")
print("But with mean-reversion, it stabilizes rather than exploding.")

In [ ]:
# Heatmap view
fig = plotter.heatmap(result, title='PDF Heatmap: Uncertainty Grows Over Time')
plt.show()

---

## Step 5: Compute E[X] - Expected Value Over Time

From the PDF, we compute the **expected value** at each time point.

This is what a point-estimate model would predict. But we have the full distribution.

In [ ]:
# Compute E[X] at each time point
times = np.linspace(0, 60, 61)
expected_values = [nig.mean(t, params_evolving) for t in times]
std_devs = [np.sqrt(nig.variance(t, params_evolving)) for t in times]

expected_values = np.array(expected_values)
std_devs = np.array(std_devs)

# Plot E[X] with confidence bands
plt.figure(figsize=(12, 6))

plt.plot(times, expected_values, 'b-', lw=2.5, label='E[X] (expected return)')
plt.fill_between(times, 
                 expected_values - 1.96*std_devs, 
                 expected_values + 1.96*std_devs, 
                 alpha=0.2, color='blue', label='95% CI')
plt.fill_between(times, 
                 expected_values - std_devs, 
                 expected_values + std_devs, 
                 alpha=0.3, color='blue', label='68% CI (±1σ)')

plt.axhline(0, color='gray', ls='--')
plt.xlabel('Days Ahead')
plt.ylabel('Expected Return (%)')
plt.title('Expected Value Over Time with Uncertainty Bands')
plt.legend(loc='upper right')
plt.grid(alpha=0.3)
plt.xlim(0, 60)
plt.show()

print(f"E[X] at t=0:  {expected_values[0]:.4f}%")
print(f"E[X] at t=30: {expected_values[30]:.4f}%")
print(f"E[X] at t=60: {expected_values[60]:.4f}%")
print()
print("A point-estimate model gives you ONLY the blue line.")
print("Distributional regression gives you the full uncertainty bands.")

---

## Step 6: Risk Measures and Decision Utilities

With the full distribution, we compute **risk measures** that point predictions can't provide:

| Measure | Question Answered |
|---------|------------------|
| **VaR** | What's the loss threshold at 5% probability? |
| **CVaR** | If we're in the worst 5%, what's the expected loss? |
| **Kelly** | What fraction of capital should we bet? |
| **P(X > k)** | What's the probability of exceeding threshold k? |

In [ ]:
# Risk measures at t=0 (current forecast)
var_95 = tpdf.var(nig, fitted_params, alpha=0.05)
var_99 = tpdf.var(nig, fitted_params, alpha=0.01)
cvar_95 = tpdf.cvar(nig, fitted_params, alpha=0.05, n_samples=50000)
cvar_99 = tpdf.cvar(nig, fitted_params, alpha=0.01, n_samples=50000)
kelly = tpdf.kelly_fraction(nig, fitted_params)

print("Risk Measures (from fitted distribution):")
print(f"  VaR 95%:  {var_95:7.2f}% (5% chance of losing more)")
print(f"  VaR 99%:  {var_99:7.2f}% (1% chance of losing more)")
print(f"  CVaR 95%: {cvar_95:7.2f}% (expected loss in worst 5%)")
print(f"  CVaR 99%: {cvar_99:7.2f}% (expected loss in worst 1%)")
print()
print(f"Position Sizing:")
print(f"  Full Kelly:    {kelly*100:6.2f}% of capital")
print(f"  Half Kelly:    {kelly*50:6.2f}% (safer)")

In [ ]:
# Visualize VaR and CVaR on the distribution
x = np.linspace(-40, 40, 1000)
pdf = nig.pdf(x, 0, fitted_params)

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(x, pdf, 'b-', lw=2.5, label='Forecast PDF')
ax.axvline(-var_95, color='orange', ls='--', lw=2.5, label=f'VaR 95% = {var_95:.1f}%')

# Shade CVaR region (tail)
tail_mask = x <= -var_95
ax.fill_between(x[tail_mask], pdf[tail_mask], alpha=0.3, color='red',
                label=f'CVaR region (E = {cvar_95:.1f}%)')

ax.axvline(0, color='gray', ls=':')
ax.set_xlabel('Return (%)')
ax.set_ylabel('Density')
ax.set_title('Risk Measures from Distributional Forecast')
ax.legend(loc='upper right')
ax.grid(alpha=0.3)
ax.set_xlim(-40, 40)
plt.tight_layout()
plt.show()

In [ ]:
# Probability queries
print("Probability Queries:")
print(f"  P(return > 0%):    {tpdf.prob_greater_than(nig, fitted_params, 0)*100:5.1f}%")
print(f"  P(return > 10%):   {tpdf.prob_greater_than(nig, fitted_params, 10)*100:5.1f}%")
print(f"  P(return < -10%):  {tpdf.prob_less_than(nig, fitted_params, -10)*100:5.1f}%")
print(f"  P(-5% < r < 5%):   {tpdf.prob_between(nig, fitted_params, -5, 5)*100:5.1f}%")

In [ ]:
# Risk measures OVER TIME (how do they evolve?)
times_risk = [0, 10, 20, 30, 60]

print("Risk Measures Over Time (with mean-reverting volatility):")
print(f"{'t':>4} | {'VaR 95%':>10} | {'CVaR 95%':>10} | {'Std Dev':>10}")
print("-" * 45)

for t in times_risk:
    var_t = tpdf.var(nig, params_evolving, alpha=0.05, t=t)
    cvar_t = tpdf.cvar(nig, params_evolving, alpha=0.05, t=t, n_samples=20000)
    std_t = np.sqrt(nig.variance(t, params_evolving))
    print(f"{t:>4} | {var_t:>10.2f}% | {cvar_t:>10.2f}% | {std_t:>10.4f}")

---

## Step 7: Evaluate with Proper Scoring Rules

How do you know if your distributional forecast is good? Use **proper scoring rules**.

**CRPS (Continuous Ranked Probability Score)**:
- Generalizes MAE to probabilistic forecasts
- Lower is better
- Same units as the target variable

Reference: Gneiting & Raftery (2007). *Strictly Proper Scoring Rules.* JASA.

In [ ]:
# Evaluate on held-out test set
rng = np.random.default_rng(42)

# CRPS for NIG forecast
crps_nig = [tpdf.crps(nig, fitted_params, y, n_samples=5000, rng=rng) for y in test_returns]

# CRPS for Normal baseline (same mean, std)
mu_baseline = np.mean(train_returns)
sigma_baseline = np.std(train_returns)
crps_normal = [tpdf.crps_normal(y, mu_baseline, sigma_baseline) for y in test_returns]

print(f"Out-of-Sample Evaluation (n={len(test_returns)} test observations)")
print(f"")
print(f"CRPS (lower is better):")
print(f"  Normal baseline: {np.mean(crps_normal):.5f}")
print(f"  NIG forecast:    {np.mean(crps_nig):.5f}")

improvement = (np.mean(crps_normal) - np.mean(crps_nig)) / np.mean(crps_normal) * 100
print(f"  Improvement:     {improvement:+.1f}%")

In [ ]:
# Visualize CRPS comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of CRPS scores
axes[0].hist(crps_normal, bins=30, alpha=0.6, label='Normal', color='green')
axes[0].hist(crps_nig, bins=30, alpha=0.6, label='NIG', color='blue')
axes[0].axvline(np.mean(crps_normal), color='green', ls='--', lw=2)
axes[0].axvline(np.mean(crps_nig), color='blue', ls='--', lw=2)
axes[0].set_xlabel('CRPS Score')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of CRPS Scores')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Scatter: NIG vs Normal CRPS per observation
axes[1].scatter(crps_normal, crps_nig, alpha=0.5, s=20)
max_val = max(max(crps_normal), max(crps_nig))
axes[1].plot([0, max_val], [0, max_val], 'r--', lw=2, label='y=x (equal)')
axes[1].set_xlabel('CRPS (Normal)')
axes[1].set_ylabel('CRPS (NIG)')
axes[1].set_title('Per-Observation CRPS: Points Below Line = NIG Wins')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Count wins
nig_wins = sum(n < g for n, g in zip(crps_nig, crps_normal))
print(f"NIG beats Normal on {nig_wins}/{len(test_returns)} observations ({100*nig_wins/len(test_returns):.1f}%)")

---

## Summary: The Complete Pipeline

```
┌─────────────────┐     ┌──────────────────┐     ┌─────────────────┐
│   Raw Data      │ ──▶ │ Feature Extract  │ ──▶ │ Model Training  │
│   (returns)     │     │ (rolling coefs)  │     │ (fit NIG/MLE)   │
└─────────────────┘     └──────────────────┘     └────────┬────────┘
                                                          │
                                                          ▼
┌─────────────────┐     ┌──────────────────┐     ┌─────────────────┐
│   Decisions     │ ◀── │   Risk Measures  │ ◀── │ Time-Evolving   │
│ (Kelly, trades) │     │ (VaR, CVaR, E[X])│     │     PDF         │
└─────────────────┘     └──────────────────┘     └─────────────────┘
         │
         ▼
┌─────────────────┐
│   Evaluation    │
│ (CRPS scoring)  │
└─────────────────┘
```

### What We Demonstrated

| Step | What | Why |
|------|------|-----|
| 1 | Load real data | Ground truth for validation |
| 2 | Extract rolling coefficients | Features for model |
| 3 | Fit NIG via MLE | Learn distribution parameters |
| 4 | Construct time-evolving PDF | Uncertainty grows over horizon |
| 5 | Compute E[X] | Point estimate with uncertainty bands |
| 6 | Risk measures | VaR, CVaR, Kelly, probabilities |
| 7 | CRPS evaluation | Proper scoring on held-out data |

### Key Results

- **NIG fits real returns better than Normal** (captures heavy tails)
- **Risk measures directly from forecast** (not possible with point estimates)
- **NIG improves CRPS** over Normal baseline on out-of-sample data

### References

- Barndorff-Nielsen (1997). Normal Inverse Gaussian Distributions. *Scand. J. Stat.*
- Bollerslev (1986). Generalized Autoregressive Conditional Heteroskedasticity. *J. Econometrics.*
- Gneiting & Raftery (2007). Strictly Proper Scoring Rules. *JASA.*
- Kelly (1956). A New Interpretation of Information Rate. *Bell System Tech. J.*